### Plot effect of polymorphisms on entry and escape
Read in csv that contains all variable sites calculated from find_variable_sites_from_alignments and plot how each of those mutations affects entry and escape.

In [ ]:
import pandas as pd
import altair as alt
import httpimport
import numpy as np

_ = alt.data_transformers.disable_max_rows()

In [ ]:
# Import custom altair theme from remote github using httpimport module
def import_theme_new():
    with httpimport.github_repo("bblarsen-sci", "altair_themes", "main"):
        import main_theme

        @alt.theme.register("custom_theme", enable=True)
        def custom_theme():
            return main_theme.main_theme()

import_theme_new()

In [ ]:
# Input data
DMS_data = pd.read_csv(snakemake.input.entry_df)

hendra_poly_df = pd.read_csv(snakemake.input.hendra_poly_df)
nipah_poly_df = pd.read_csv(snakemake.input.nipah_poly_df)

escape_df = pd.read_csv(snakemake.input.escape_df)

# Params
antibody_order = snakemake.params.antibody_order

### Paths for running manually

In [ ]:
#DMS_data = pd.read_csv('../../results/filtered_data/cell_entry/Nipah_F_func_effects_filtered.csv')
#hendra_poly_df = pd.read_csv('../../results/site_differences/hendra_F_variable_sites_detailed.csv')
#nipah_poly_df = pd.read_csv('../../results/site_differences/nipah_F_variable_sites_detailed.csv')
#escape_df = pd.read_csv('../../results/filtered_data/antibody_escape/combined/all_antibodies_escape_filtered.csv')
#antibody_order = ["12B2", "2D3", "4H3", "1A9", "1F2", "2B12"]


In [ ]:
# Merge polymorphism data with DMS functional effects
def merge_seq_polymorphisms_with_functional_effects(DMS_data, seq_polymorphism_data):
    seq_polymorphism_data = seq_polymorphism_data.query("site >= 29 and site <= 481")
    merged_df = pd.merge(
        seq_polymorphism_data, DMS_data, how="left", on=["site", "wildtype", "mutant"]
    )
    return merged_df


hendra_merged = merge_seq_polymorphisms_with_functional_effects(
    DMS_data, hendra_poly_df
)
nipah_merged = merge_seq_polymorphisms_with_functional_effects(DMS_data, nipah_poly_df)

combined = pd.concat(
    [hendra_poly_df.assign(virus="Hendra"), nipah_poly_df.assign(virus="Nipah")],
    ignore_index=True,
).query("site >= 29 and site <= 481")
display(combined)

In [ ]:
# find how many rows have NaN values in the effect column
display(nipah_merged.sort_values('effect'))
display(hendra_merged.sort_values('effect'))

In [ ]:
# make a bar chart of the distribution of functional effects for each hendra and nipah mutation
def make_cell_entry_bar_chart(df):
    bar = (
        alt.Chart(df)
        .mark_rect(fill="#b8b0ac", stroke="black", strokeWidth=1)
        .encode(
            x=alt.X(
                "x1:Q",
                title=["Effects of mutations", "on Nipah F cell entry"],
                scale=alt.Scale(domain=[-4, 1]),
                axis=alt.Axis(tickCount=4),
            ),
            x2="x2:Q",
            y=alt.Y("count()", title="Mutation count", axis=alt.Axis(tickCount=4)),
        )
        .transform_bin(["x1", "x2"], field="effect", bin=alt.Bin(step=0.1))
    )
    return bar


# make the chart for nipah
nipah_mut_histogram = make_cell_entry_bar_chart(nipah_merged)

# make the chart for hendra
hendra_mut_histogram = make_cell_entry_bar_chart(hendra_merged)

# Combine the two histograms into a single chart
combined_histogram = alt.hconcat(
    nipah_mut_histogram, hendra_mut_histogram
).resolve_scale(x="shared", y="shared")
combined_histogram.display()


In [ ]:
# Save the chart
combined_histogram.save(snakemake.output.cell_entry_plot_svg)
combined_histogram.save(snakemake.output.cell_entry_plot_png, ppi=300)


### Antibody Escape
Now do the same thing above from the functional effects data but with the antibody escape data.

In [ ]:
# setup dataframe with mutation column
escape_df["mutation"] = (
    escape_df["wildtype"].astype(str)
    + escape_df["site"].astype(str)
    + escape_df["mutant"]
)

merged_escape = pd.merge(
    combined,
    escape_df,
    how="outer",
    on=["site", "wildtype", "mutant"],
)
# check to see which mutations are missing from the escape data

# replace empty strings
merged_escape["virus"] = merged_escape["virus"].fillna("Not observed")
display(merged_escape)

In [ ]:
# plot escape data
nipah_chart = (
    alt.Chart(merged_escape.query("escape_mean > 0 & virus != 'Not observed'"))
    .mark_point(filled=True, strokeWidth=0.5, stroke="black", size=75, opacity=1)
    .encode(
        x=alt.X(
            "virus:O",
            title=None,
            sort=["Not observed", "Nipah", "Hendra"],
            axis=alt.Axis(title=None, labels=False, ticks=True),
        ),
        y=alt.Y("escape_mean", title="Escape", axis=alt.Axis(tickCount=4, grid=True)),
        xOffset="jitter:Q",
        color=alt.Color(
            "virus:N",
            title=None,
            sort=["Not observed", "Nipah", "Hendra"],
            scale=alt.Scale(range=["#1C507B", "#F57F20"]),
        ),
        column=alt.Column(
            "antibody",
            title="Antibody",
            spacing=0,
            sort=antibody_order,
            header=alt.Header(
                titleOrient="bottom",
                labelOrient="bottom",
                titlePadding=0,
                labelPadding=5,
            ),
        ),
        tooltip=["antibody", "mutation", "escape_mean"],
    )
    .transform_calculate(jitter="sqrt(-2*log(random()))*cos(2*PI*random())")
    .properties(width=40, height=75)
)

display(nipah_chart)
nipah_chart.save('../../nipah_escape_polymorphisms.png', ppi=300)
nipah_chart.save('../../nipah_escape_polymorphisms.svg')

In [ ]:
# Save the chart
nipah_chart.save(snakemake.output.escape_plot_svg)
nipah_chart.save(snakemake.output.escape_plot_png, ppi=300)

In [ ]:
#ic50_data = pd.read_csv('../../results/ic50_data/ic50_fold_changes_hendra.csv')
#ic50_data["at_threshold"] = np.where(ic50_data["ic50_mutant"] == 10, "yes", "no")
#ic50_data["ic50_fold"] = ic50_data["ic50_fold"].round(1)
#display(ic50_data)

In [ ]:
# Plot IC50 values for unmutated and mutant for each antibody

# Base chart
#ic50_chart_base = (
#    alt.Chart(ic50_data)
#    .mark_point(
#        filled=True, strokeWidth=1, stroke="black", size=75, opacity=1
#    )
#    .encode(x=alt.X("antibody:O", title=None, sort=antibody_order))
#)
#
## Unmutated and mutant points
#unmutated = ic50_chart_base.encode(
#    y=alt.Y("ic50_unmutated:Q", title=None), color=alt.value("#1C507B")
#)
#mutant = ic50_chart_base.encode(
#    y=alt.Y("ic50_mutant:Q", title=None), color=alt.value("#F57F20")
#)
#
## Gray lines connecting unmutated and mutant points
#gray_lines = (
#    alt.Chart(ic50_data)
#    .mark_rule(color="#b8b0ac", opacity=0.75)
#    .encode(
#        x=alt.X(
#            "antibody:O",
#            title=None,
#            sort=antibody_order,
#            axis=alt.Axis(labelAngle=0),
#        ),
#        y=alt.Y(
#            "ic50_unmutated:Q",
#            axis=alt.Axis(tickCount=3, grid=False, labelAngle=0),
#            scale=alt.Scale(type="log"),
#        ),
#        y2=alt.Y2("ic50_mutant:Q"),
#    )
#)
#
## Horizontal line at y=10 (limit of detection)
#rule = (
#    alt.Chart()
#    .mark_rule(color="#b8b0ac", strokeDash=[2, 2], opacity=0.5)
#    .encode(y=alt.datum(10))
#)
#
## Text labels for fold change
#text = (
#    alt.Chart(ic50_data)
#    .mark_text(dy=-10, baseline="bottom", align="center")
#    .encode(
#        x=alt.X(
#            "antibody:O",
#            title=None,
#            sort=antibody_order,
#            axis=alt.Axis(labelAngle=0),
#        ),
#        y=alt.datum(10),
#        text=alt.Text("ic50_fold:N"),
#    )
#)
#
## Combine all layers
#combined_IC50_change = alt.layer(rule, gray_lines, unmutated, mutant, text).properties(
#    width=240, height=50
#)
#
#display(combined_IC50_change)
##combined_IC50_change.save("../../ic50_chart.png", ppi=300)
##combined_IC50_change.save("../../ic50_chart.svg")
#